In [ ]:
# Khai báo các thư viện cần thiết
import numpy as np
import random
import pickle
from collections import deque
import os

In [ ]:
# Khai báo các hằng số sử dụng
class Maze:
    WALL = -1
    PATH = 0
    UNEXPLORED = 1
    GOAL = 2
    AGENT_POSITION = 3

In [ ]:
# Môi trường Mê cung

class MazeEnv():
    # Môi trường mê cung
    # Maze: Mê cung được tạo ra ngẫu nhiên với các ô đường và tường
    # Với các ô đường được đánh dấu là 0 và các ô tường được đánh dấu là -1, ô đích được đánh dấu là 2
    # Goal_position: Vị trí đích trong mê cung

    def __init__(self, maze_size, local_obs_size = 3, max_steps = 15, path_percent = 70):
        """
        Khởi tạo môi trường Mê cung.

        Args:
        - maze_size (int): Kích thước của mê cung (ví dụ: 50x50).
        - max_steps (int): Số bước tối đa cho mỗi tập.
        - path_percent (int): Tỷ lệ phần trăm ô đường trong mê cung (0-100).
        """
        # Đặt các thông số của môi trường
        super(MazeEnv, self).__init__()  # Kế thừa từ gym.Env
        
        self.maze_size = maze_size
        self.max_steps = max_steps
        self.path_percent = path_percent
        self.local_obs_size = local_obs_size
        
        self.maze = np.ones((maze_size, maze_size), dtype=int)  # Mặc định là chưa biết (1)
        self.base_position = (maze_size // 6 - 1, maze_size // 6 - 1)  # Vị trí khởi đầu của tác tử
        self.goal_position = (maze_size * 5 // 6, maze_size * 5 // 6)  # Đích cố định tại giữa khu vực đích

    def get_neighbors(self, x, y, max_size = 0):
        """
        Lấy danh sách các ô lân cận.
        """
        if max_size == 0:
            max_size = self.maze_size
        neighbors = []
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < max_size and 0 <= ny < max_size:
                neighbors.append((nx, ny))
        return neighbors
        
    # Phương thức lấy phần thưởng của 1 bước đi
    def get_reward(self, p1, p2, discored_score):

        # Khởi tạo biến phần thưởng
        reward = 0

        if self.maze[p2] == Maze.WALL:
            reward -= 2 * self.maze_size  # Phạt lớn khi va chạm với tường
        elif p2 == self.goal_position:
            reward += 3 * self.maze_size  # Thưởng lớn khi đến đích
        else:
            # Tính khoảng cách Manhattan từ vị trí hiện tại tới đích
            current_distance = abs(p2[0] - self.goal_position[0]) + abs(p2[1] - self.goal_position[1])
            
            # Tính khoảng cách trước đó
            previous_distance = abs(p1[0] - self.goal_position[0]) + abs(p1[1] - self.goal_position[1])

            # Tính khoảng cách đã đi được
            moved_distance = 1 + abs(p2[0] - self.base_position[0]) + abs(p2[1] - self.base_position[1])

            # Tăng thưởng nếu đến gần đích hơn
            if discored_score == Maze.PATH:  # Nếu ô chưa được khám phá
                if current_distance < previous_distance:
                    reward += 2 * self.maze_size // current_distance  # Thưởng khi di chuyển đến gần đích hơn
                else:
                    reward -= 2 * self.maze_size // moved_distance # Phạt nếu tác tử đi xa hơn
            else:
                # Phạt khi đi vào ô đã khám phá
                reward += discored_score * self.maze_size // current_distance + discored_score * self.maze_size // moved_distance
        
        return reward
    
    # Phương thức để kiểm tra tính hợp lệ của một vị trí
    def valid_check(self, p1):
        if 0 <= p1[0] < self.maze_size and 0 <= p1[1] < self.maze_size and self.maze[p1] != Maze.WALL:
            return True
        return False

    def export_data(self, start_position, gamma, filename):
        def get_position(relative_position):
            return (relative_position[0] + start_position[0] - self.local_obs_size, relative_position[1] + start_position[1] - self.local_obs_size)
        # Tạo dữ liệu keys
        minx = max(0, start_position[0] - self.local_obs_size)
        maxx = min(self.maze_size, start_position[0] + self.local_obs_size + 1)
        miny = max(0, start_position[1] - self.local_obs_size)
        maxy = min(self.maze_size, start_position[1] + self.local_obs_size + 1)
        key_data = np.zeros((self.local_obs_size * 2 + 1, self.local_obs_size * 2 + 1), dtype=int)
        key_data.fill(Maze.WALL)
        key_data[minx - start_position[0] + self.local_obs_size:maxx - start_position[0] + self.local_obs_size, miny - start_position[1] + self.local_obs_size:maxy - start_position[1] + self.local_obs_size] = self.maze[minx:maxx, miny:maxy]
        
        values = [-4, -3, -2, -1, 0]
        probabilities = [0.005, 0.02, 0.05, 0.1, 0.825]
        key_data = 5 * key_data
        for i in range(self.local_obs_size * 2 + 1):
            for j in range(self.local_obs_size * 2 + 1):
                if key_data[i][j] == Maze.PATH:
                    key_data[i][j] = random.choices(values, probabilities)[0]
                
        p = self.get_neighbors(self.local_obs_size, self.local_obs_size)
        values = []
        for position in p:
            if key_data[position] == 5 * Maze.WALL:
                values.append(-2 * self.maze_size)
                continue
            queue = deque([((self.local_obs_size, self.local_obs_size), position)])
            visited = set()
            visited.add(((self.local_obs_size, self.local_obs_size), position))
            visited.add((p[0], (self.local_obs_size, self.local_obs_size)))
            visited.add((p[1], (self.local_obs_size, self.local_obs_size)))
            visited.add((p[2], (self.local_obs_size, self.local_obs_size)))
            visited.add((p[3], (self.local_obs_size, self.local_obs_size)))
            
            reward_array = np.zeros((self.local_obs_size * 2 + 1, self.local_obs_size * 2 + 1), dtype=float)
            reward_array.fill(-10000)
            reward_array[self.local_obs_size, self.local_obs_size] = 0

            step_count = 0
            neighbors_count = 1

            while queue:
                new_neighbors_count = 0
                for _ in range(neighbors_count):
                    p1, p2 = queue.popleft()  # Lấy phần tử đầu tiên trong deque
                    reward_array[p2] = max(reward_array[p1] + gamma ** step_count * float(self.get_reward(get_position(p1), get_position(p2), key_data[p2])), reward_array[p2]) 

                    # Thêm các ô lân cận vào deque
                    neighbors = self.get_neighbors(p2[0], p2[1], self.local_obs_size * 2 + 1)
                    for nx, ny in neighbors:
                        if key_data[nx, ny] != 5 * Maze.WALL and (p2, (nx, ny)) not in visited:
                            queue.append((p2, (nx, ny)))  # Thêm vào cuối deque
                            visited.add((p2, (nx, ny)))  # Đánh dấu ô đã được khám phá
                            visited.add(((nx, ny), p2))  # Đánh dấu ô đã được khám phá
                            new_neighbors_count += 1
                neighbors_count = new_neighbors_count
                step_count += 1

            max_value = - 10000
            for i in range(self.local_obs_size * 2 + 1):
                if reward_array[i][0] > max_value:
                    max_value = reward_array[i][0]
                if reward_array[i][self.local_obs_size * 2] > max_value:
                    max_value = reward_array[i][self.local_obs_size * 2]
            for j in range(self.local_obs_size * 2 + 1):
                if reward_array[0][j] > max_value:
                    max_value = reward_array[0][j]
                if reward_array[self.local_obs_size * 2][j] > max_value:
                    max_value = reward_array[self.local_obs_size * 2][j]
            values.append(max_value)
        max_value = max(values)
        for i in range(len(values)):
            if values[i] == -2 * self.maze_size:
                values[i] += gamma * max_value
        with open(filename, 'ab') as f:
            pickle.dump((key_data, values), f)

In [ ]:
maze_size = 30
local_obs_size = 3
max_steps = 15
path_percent = 70
env = MazeEnv(maze_size, local_obs_size, max_steps, path_percent)

max_episode = 1000
gamma = 0.8
for episode in range(max_episode):
    env.generate_maze()
    for i in range(maze_size):
        for j in range(maze_size):
            if env.maze[i][j] == Maze.PATH:
                env.export_data((i, j), gamma, 'dataset\\dt' + str(i) + '_' + str(j) + '.pkl')
    print('Episode:', episode + 1, 'completed.')